[IPython Notebook](alternate_backends.ipynb) |  [Python Script](alternate_backends.py)

Advanced: Alternate Backends
============================

Setup
-----------------------------

As always, let's do imports and initialize a logger and a new Bundle.  See [Building a System](building_a_system.html) for more details.

In [1]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger(clevel='INFO')

b = phoebe.default_binary()

/usr/local/lib/python2.7/dist-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


/usr/local/lib/python2.7/dist-packages/astropy/units/quantity.py:732: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  return super(Quantity, self).__eq__(other)


And we'll attach some dummy datasets.  See [Datasets](datasets.html) for more details.

In [2]:
b.add_dataset('orb', times=np.linspace(0,10,1000), dataset='orb01', component=['primary', 'secondary'])

<ParameterSet: 2 parameters | components: primary, secondary>

In [3]:
b.add_dataset('lc', times=np.linspace(0,10,1000), dataset='lc01')

<ParameterSet: 11 parameters | kinds: lc, lc_dep>

Available Backends
----------------------------

See the [Compute Tutorial](./compute) for details on adding compute options and using them to create synthetic models.


### PHOEBE 1.0 Legacy

For more details, see [Comparing PHOEBE 2.0 vs PHOEBE Legacy](../examples/legacy)

In [4]:
b.add_compute('legacy', compute='legacycompute')
print b['legacycompute']

ParameterSet: 10 parameters
  store_mesh@legacycompute@co...: False
  morphology@legacycompute@co...: Detached binary
    cindex@legacycompute@compute: False
  refl_num@legacycompute@compute: 1
        ie@legacycompute@compute: False
   enabled@legacycompute@compute: True
  atm@primary@legacycompute@c...: kurucz
  atm@secondary@legacycompute...: kurucz
  gridsize@primary@legacycomp...: 40
  gridsize@secondary@legacyco...: 40


### Photodynam

For more details, see [Comparing PHOEBE 2.0 vs Photodynam (Binary)](../examples/photodynam) and [Comparing PHOEBE 2.0 vs Photodynam (Hierarchical Triple)](../examples/hierarchical_triple)

In [5]:
b.add_compute('photodynam', compute='photodynamcompute')
print b['photodynamcompute']

ParameterSet: 4 parameters
  stepsize@photodynamcompute@...: 0.01
  orbiterror@photodynamcomput...: 1e-20
  enabled@lc01@photodynamcomp...: True
  enabled@orb01@photodynamcom...: True


### JKTEBOP

For more details, see [Comparing PHOEBE 2.0 vs JKTEBOP](../examples/jktebop)

In [6]:
b.add_compute('jktebop', compute='jktebopcompute')
print b['jktebopcompute']

ParameterSet: 2 parameters
  ringsize@jktebopcompute@com...: 5.0 deg
  enabled@lc01@jktebopcompute...: True


Using Alternate Backends
---------------------------------


### Adding Compute Options

Adding a set of compute options for an alternate backend is just as easy as for the PHOEBE backend.  Simply provide the function or name of the function in phoebe.parameters.compute that points to the parameters for that backend.

Here we'll add the default PHOEBE backend as well as photodynam by Josh Carter.  Note that in order to use an alternate backend, that backend must be installed on your machine.

In [7]:
b.add_compute('phoebe', compute='phoebebackend')

<ParameterSet: 20 parameters | datasets: lc01, orb01>

In [8]:
print b['phoebebackend']

ParameterSet: 20 parameters
  dynamics_method@phoebebacke...: keplerian
      ltte@phoebebackend@compute: False
      refl@phoebebackend@compute: False
  store_mesh@phoebebackend@co...: False
  mesh_offset@phoebebackend@c...: True
  eclipse_method@phoebebacken...: native
  enabled@lc01@phoebebackend@...: True
  enabled@orb01@phoebebackend...: True
  mesh_method@primary@phoebeb...: marching
  mesh_method@secondary@phoeb...: marching
  delta@primary@phoebebackend...: 0.1
  delta@secondary@phoebebacke...: 0.1
  maxpoints@primary@phoebebac...: 100000
  maxpoints@secondary@phoebeb...: 100000
  distortion_method@primary@p...: roche
  distortion_method@secondary...: roche
  atm@primary@phoebebackend@c...: ck2004
  atm@secondary@phoebebackend...: ck2004
  lc_method@phoebebackend@com...: numerical
  fti_method@phoebebackend@co...: None


In [9]:
b.add_compute('photodynam', compute='pdbackend')

<ParameterSet: 4 parameters | datasets: lc01, orb01>

In [10]:
print b['pdbackend']

ParameterSet: 4 parameters
      stepsize@pdbackend@compute: 0.01
    orbiterror@pdbackend@compute: 1e-20
  enabled@lc01@pdbackend@compute: True
  enabled@orb01@pdbackend@com...: True


### Running Compute

Nothing changes for running compute - simply provide the compute tag for those options.  Do note, however, that not all backends support all dataset types.

In [12]:
b.set_value_all('pblum_ref', 'self')
# TODO: remove this once other backends support coupled pblums

In [13]:
b.run_compute('pdbackend', model='pdresults')

<ParameterSet: 16 parameters | kinds: orb, lc>

### Running Fitting

COMING SOMEDAY

Running Multiple Backends Simultaneously
---------------------------------------------------

Running multiple backends simultaneously is just as simple as running the PHOEBE backend with multiple sets of compute options (see [Compute](compute.html)).

We just need to make sure that each dataset is only enabled for one (or none) of the backends that we want to use, and then send a list of the compute tags to run_compute.  Here we'll use the PHOEBE backend to compute orbits and the photodynam backend to compute light curves.

In [14]:
b.set_value_all('enabled@lc01@phoebebackend', False)
b.set_value_all('enabled@orb01@pdbackend', False)
print b['enabled']

ParameterSet: 8 parameters
  enabled@lc01@legacycompute@...: True
  enabled@lc01@photodynamcomp...: True
  enabled@orb01@photodynamcom...: True
  enabled@lc01@jktebopcompute...: True
  enabled@lc01@phoebebackend@...: False
  enabled@orb01@phoebebackend...: True
  enabled@lc01@pdbackend@compute: True
  enabled@orb01@pdbackend@com...: False


In [15]:
b.run_compute(['phoebebackend', 'pdbackend'], model='mixedresults')

<ParameterSet: 16 parameters | kinds: orb, lc>

The parameters inside the returned model even remember which set of compute options (and therefore, in this case, which backend) were used to compute them.

In [16]:
print b['mixedresults'].computes

['phoebebackend', 'pdbackend']


In [17]:
b['mixedresults@phoebebackend'].datasets

['orb01']

In [18]:
b['mixedresults@pdbackend'].datasets

['lc01']